In [1]:
import torch
import mdtraj as md
import nglview as nv

In [ ]:
molecule = "2jof"

gro_path = f"./{molecule.upper()}/folded.gro"
traj = md.load(gro_path)
traj.center_coordinates()
view = nv.show_mdtraj(traj)
view.add_representation("licorice", selection="protein")

ca_atoms = [atom for atom in traj.topology.atoms if atom.name == "CA"]
ca_indices = [atom.index + 1 for atom in ca_atoms]
print("Alpha carbon indices:", ca_indices)

view

Alpha carbon indices: [5, 17, 27, 48, 58, 75, 99, 118, 128, 140, 147, 162, 168, 179, 190, 197, 229, 243, 257, 263]


NGLWidget()

In [ ]:
cnt = 1
for i in range(len(ca_indices)):
    for j in range(i+1, len(ca_indices)):
        print(f"d{cnt}: DISTANCE ATOMS={ca_indices[i]},{ca_indices[j]}")
        cnt += 1

Alpha carbon indices: [5, 17, 27, 48, 58, 75, 99, 118, 128, 140, 147, 162, 168, 179, 190, 197, 229, 243, 257, 263]
d1: DISTANCE ATOMS=5,17
d2: DISTANCE ATOMS=5,27
d3: DISTANCE ATOMS=5,48
d4: DISTANCE ATOMS=5,58
d5: DISTANCE ATOMS=5,75
d6: DISTANCE ATOMS=5,99
d7: DISTANCE ATOMS=5,118
d8: DISTANCE ATOMS=5,128
d9: DISTANCE ATOMS=5,140
d10: DISTANCE ATOMS=5,147
d11: DISTANCE ATOMS=5,162
d12: DISTANCE ATOMS=5,168
d13: DISTANCE ATOMS=5,179
d14: DISTANCE ATOMS=5,190
d15: DISTANCE ATOMS=5,197
d16: DISTANCE ATOMS=5,229
d17: DISTANCE ATOMS=5,243
d18: DISTANCE ATOMS=5,257
d19: DISTANCE ATOMS=5,263
d20: DISTANCE ATOMS=17,27
d21: DISTANCE ATOMS=17,48
d22: DISTANCE ATOMS=17,58
d23: DISTANCE ATOMS=17,75
d24: DISTANCE ATOMS=17,99
d25: DISTANCE ATOMS=17,118
d26: DISTANCE ATOMS=17,128
d27: DISTANCE ATOMS=17,140
d28: DISTANCE ATOMS=17,147
d29: DISTANCE ATOMS=17,162
d30: DISTANCE ATOMS=17,168
d31: DISTANCE ATOMS=17,179
d32: DISTANCE ATOMS=17,190
d33: DISTANCE ATOMS=17,197
d34: DISTANCE ATOMS=17,229
d35: D

In [ ]:
print("ARG=" + ",".join(f"d{i+1}" for i in range(cnt)))